In [1]:
import seaborn as sns
import metapack as mp
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display 

%matplotlib inline
sns.set_context('notebook')
mp.jupyter.init()


In [2]:
pkg = mp.jupyter.open_package()
#pkg = mp.jupyter.open_source_package()
pkg

# San Diego Pedestrian Counts
`cityiq.io-pedestrians-san_diego-1` Last Update: 2019-06-09T15:57:11

_Counts of pedestrian traffic from the San Diego CityIQ system_


This dataset is pedestrian directions and counts for walkwaks in the San Diego CityIQ system. 

Before generating this package, you must have scraped all of the events to be included in the package. After creating a ``.city-iq.yaml`` configuration file, run: 

    $ ciq_events -s ped # Scrape the events
    $ ciq_aggregate ped # Convert JSON to CSV
    
The first process ``ciq_events`` may run for several days and consume up to 300G of disk space. The second process, ``ciq_aggregate`` may run for a few hours and consume a few more gigabytes of disk space.


 
 
## Contacts

* **Wrangler** [Eric Busboom](mailto:eric@civicknowledge.com), [Civic Knowledge](http://civicknowledge.com)
 
## Resources

* ** [pedestrians](data/pedestrians.csv)**. Pedestrians

In [3]:
ped = pkg.resource('pedestrians').dataframe()
ped.head()

,timestamp,locationuid,direction,speed,count
0,2018-11-01 08:00:00,004361eb,165,2.641,1
1,2018-11-01 09:15:00,004361eb,165,2.641,1
2,2018-11-01 11:45:00,004361eb,165,2.641,1
3,2018-11-01 11:45:00,004361eb,345,2.641,1
4,2018-11-01 12:45:00,004361eb,165,2.641,1


In [6]:
ped.timestamp.min(), ped.timestamp.max()

(Timestamp('2018-08-01 06:45:00'), Timestamp('2019-06-01 00:00:00'))

In [4]:
obj = mp.open_package('index:cityiq.io-objects-san_diego')
obj

# San Diego City IQ Assets and Locations
`cityiq.io-objects-san_diego-5` Last Update: 2019-06-04T23:09:02

_All assets types extracted form the San Diego City Iq system_


These datafile are extracts of the assets and locations from the San Diego
CityIQ system. Refer to the [CityIQ developer
documentation](https://ie-cities-docs.run.aws-usw02-pr.ice.predix.io/#c_overview
 _of_general_apis.html) for details about these data records. The data are
extracted using the [cityiq Python package](https://pypi.org/project/cityiq/).
See the ExtractAssets.ipynb notebook for the extract process.

## Versions

1. Development version
2. Development version
3. First release
4. Added tract and community names
5. Rebuild with recent data, and broke out asset and location types


 
 
## Contacts

* **Wrangler** [Eric Busboom](mailto:eric@civicknowledge.com), [Civic Knowledge](http://civicknowledge.com)
 
## Resources

* ** [assets](data/assets.csv)**. All assets
* ** [locations](data/locations.csv)**. All locations: walkways, traffic lanes and parking zones
* ** [walkways](data/walkways.csv)**. Walkways
* ** [traffic_lanes](data/traffic_lanes.csv)**. Traffic lanes
* ** [parking_zones](data/parking_zones.csv)**. Parking zones
* ** [cameras](data/cameras.csv)**. Cameras
* ** [mics](data/mics.csv)**. Microphones
* ** [nodes](data/nodes.csv)**. Nodes
* ** [em_sensors](data/em_sensors.csv)**. Electro-magnetic sensors
* ** [env_sensors](data/env_sensors.csv)**. Environmental sensors
 
## References

* **[sdroads](metapack+http://library.metatab.org/sangis.org-roads-sandiego-1.zip#all_roads)**. Roads in San Diego
* **[tract_boundaries](metapack+http://library.metatab.org/sandiegodata.org-geography-2018.csv#tract_boundaries)**. San Diego tracts and communities
* **[sd_community_boundaries](metapack+http://library.metatab.org/sandiegodata.org-geography-2018.csv#sd_community_boundaries)**. San Diego tracts and communities

In [5]:
walkways = obj.resource('walkways').geoframe()

In [6]:
def tod(v):
    if 6<=v.hour<11:
        return 'morning'
    elif 11<=v.hour<13:
        return 'lunch'
    elif 13<=v.hour<17:
        return 'afternoon'
    elif 16<=v.hour<22:
        return 'evening'
    else:
        return 'night'
    
ped['tod'] = ped.timestamp.apply(tod)
ped.head()
    

,timestamp,locationuid,direction,speed,count,tod
0,2018-11-01 08:00:00,004361eb,165,2.641,1,morning
1,2018-11-01 09:15:00,004361eb,165,2.641,1,morning
2,2018-11-01 11:45:00,004361eb,165,2.641,1,lunch
3,2018-11-01 11:45:00,004361eb,345,2.641,1,lunch
4,2018-11-01 12:45:00,004361eb,165,2.641,1,lunch


In [7]:
ped['dow'] = ped.timestamp.dt.dayofweek

In [8]:
ped_tod = ped.groupby(['dow', 'tod','locationuid']).sum()
ped_tod.head()

direction        speed  count
dow tod       locationuid                               
0   afternoon 004361eb         50205   573.097000    380
              0051796c         43350   988.442133   1899
              005f90ed         64000   519.973074    524
              006bbb6e         78480   615.857205    706
              006da86e        114467  1416.982697   7663

In [9]:
walkways.head()

,locationuid,locationtype,parentlocationuid,community_name,tract_geoid,roadsegid,speed,oneway,abloaddr,abhiaddr,rd30full,geometry
0,004361eb,WALKWAY,004361eb,Downtown,14000US06073005800,40052.0,35.0,T,2100.0,2199.0,KETTNER BLVD,LINESTRING (-117.1706148251287 32.726548905684...
1,0051796c,WALKWAY,0051796c,Downtown,14000US06073005800,54946.0,20.0,F,1600.0,1699.0,STATE ST,LINESTRING (-117.1666581391174 32.722214661862...
2,005f90ed,WALKWAY,005f90ed,Downtown,14000US06073005800,54945.0,20.0,F,1700.0,1799.0,STATE ST,LINESTRING (-117.1665182431568 32.723810889902...
3,006bbb6e,WALKWAY,006bbb6e,Downtown,14000US06073005100,2809.0,20.0,B,400.0,499.0,13TH ST,LINESTRING (-117.1529442467722 32.710476055788...
4,006da86e,WALKWAY,006da86e,Downtown,14000US06073005400,40049.0,20.0,T,1300.0,1399.0,KETTNER BLVD,LINESTRING (-117.1691082585662 32.719393495624...


In [10]:
t = walkways.merge(ped, on='locationuid', how='right')
t.head()

,locationuid,locationtype,parentlocationuid,community_name,tract_geoid,roadsegid,speed_x,oneway,abloaddr,abhiaddr,rd30full,geometry,timestamp,direction,speed_y,count,tod,dow
0,004361eb,WALKWAY,004361eb,Downtown,14000US06073005800,40052.0,35.0,T,2100.0,2199.0,KETTNER BLVD,LINESTRING (-117.1706148251287 32.726548905684...,2018-11-01 08:00:00,165,2.641,1,morning,3
1,004361eb,WALKWAY,004361eb,Downtown,14000US06073005800,40052.0,35.0,T,2100.0,2199.0,KETTNER BLVD,LINESTRING (-117.1706148251287 32.726548905684...,2018-11-01 09:15:00,165,2.641,1,morning,3
2,004361eb,WALKWAY,004361eb,Downtown,14000US06073005800,40052.0,35.0,T,2100.0,2199.0,KETTNER BLVD,LINESTRING (-117.1706148251287 32.726548905684...,2018-11-01 11:45:00,165,2.641,1,lunch,3
3,004361eb,WALKWAY,004361eb,Downtown,14000US06073005800,40052.0,35.0,T,2100.0,2199.0,KETTNER BLVD,LINESTRING (-117.1706148251287 32.726548905684...,2018-11-01 11:45:00,345,2.641,1,lunch,3
4,004361eb,WALKWAY,004361eb,Downtown,14000US06073005800,40052.0,35.0,T,2100.0,2199.0,KETTNER BLVD,LINESTRING (-117.1706148251287 32.726548905684...,2018-11-01 12:45:00,165,2.641,1,lunch,3


In [11]:
t.columns

Index(['locationuid', 'locationtype', 'parentlocationuid', 'community_name',
       'tract_geoid', 'roadsegid', 'speed_x', 'oneway', 'abloaddr', 'abhiaddr',
       'rd30full', 'geometry', 'timestamp', 'direction', 'speed_y', 'count',
       'tod', 'dow'],
      dtype='object')

In [12]:
t2 = t[t.community_name == 'Downtown'].groupby(['roadsegid', 'tod','dow'])\
    .agg({
        'count': 'mean',
    }).reset_index()

In [13]:
t2.head()

,roadsegid,tod,dow,count
0,239.0,afternoon,2,5.322581
1,239.0,afternoon,3,6.419355
2,239.0,afternoon,4,6.281250
3,239.0,afternoon,5,3.541667
4,239.0,afternoon,6,2.960000


In [14]:
roads = obj.reference('sdroads').geoframe()

In [15]:
roads.head()

,id,fnode,tnode,length,roadsegid,postid,postdate,roadid,rightway,addsegdt,...,rd20name,rd20sfx,rd20full,rd30pred,rd30name,rd30sfx,rd30postd,rd30full,shape_stle,geometry
0,0,111384,110959,1284.187993,37710,fjessie,2014-11-05,41110,0,1989-02-01,...,I-5 NB,NaN,I-5 NB,NaN,INTERSTATE 5 NB,NaN,NaN,INTERSTATE 5 NB,1284.187993,LINESTRING (-117.0930208799119 32.622324029187...
1,1,110466,110965,1344.487233,135090,fjessie,2014-11-05,100403,0,1989-02-01,...,I-5 SB ON,RA,I-5 SB ON RA,NaN,INTERSTATE 5 SB ON,RAMP,NaN,INTERSTATE 5 SB ON RAMP,1344.487233,LINESTRING (-117.0961421938108 32.629118886531...
2,2,182199,182198,304.156973,220417,fjessie,2017-12-15,129283,0,2017-01-18,...,CALLE DECEO,NaN,CALLE DECEO,NaN,CALLE DECEO,NaN,NaN,CALLE DECEO,304.110177,LINESTRING (-116.9959048225822 32.597350154835...
3,3,56489,56473,314.963225,71567,FJessie,2013-04-15,19332,56,1980-04-01,...,ALCAMO,RD,ALCAMO RD,NaN,ALCAMO,RD,NaN,ALCAMO RD,314.963225,LINESTRING (-117.1548070648652 32.913428734897...
4,4,16400,16404,388.972250,162506,fjessie,2016-10-03,4951,0,NaT,...,CATALINA,AV,CATALINA AV,NaN,CATALINA,AVE,NaN,CATALINA AVE,388.972250,LINESTRING (-117.1988265137608 33.208256676586...


In [16]:
t3 = gpd.GeoDataFrame(t2.merge(roads, on='roadsegid'), geometry='geometry')

In [17]:
t4 = t3[['roadsegid','tod', 'dow','speed', 'count', 'geometry']]

In [18]:
t4.to_csv('segments.csv')

In [19]:
t4['count'].max()

71.94

In [20]:
t4[t4['count']>100]

,roadsegid,tod,dow,speed,count,geometry


In [21]:
len(t2), len(t3)

(11269, 11269)

In [22]:
sum(t3.roadsegid.isnull())

0

In [23]:
t[t.locationuid == 'f3ae56b1']

,locationuid,locationtype,parentlocationuid,community_name,tract_geoid,roadsegid,speed_x,oneway,abloaddr,abhiaddr,rd30full,geometry,timestamp,direction,speed_y,count,tod,dow


In [24]:
t4.sort_values('count', ascending = False).head(30)

,roadsegid,tod,dow,speed,count,geometry
4445,3149.0,lunch,3,20,71.940000,LINESTRING (-117.1485492625946 32.710455454220...
8448,37312.0,night,6,20,61.556021,LINESTRING (-117.1682933905995 32.722994419422...
1231,1658.0,lunch,0,20,60.039801,LINESTRING (-117.1592976256904 32.721978422488...
8447,37312.0,night,5,20,53.217050,LINESTRING (-117.1682933905995 32.722994419422...
1233,1658.0,lunch,2,20,53.088235,LINESTRING (-117.1592976256904 32.721978422488...
1237,1658.0,lunch,6,20,51.164948,LINESTRING (-117.1592976256904 32.721978422488...
1240,1658.0,morning,2,20,50.992767,LINESTRING (-117.1592976256904 32.721978422488...
10789,162992.0,lunch,1,20,50.801282,LINESTRING (-117.1555668901108 32.708354564029...
10795,162992.0,morning,0,20,48.701456,LINESTRING (-117.1555668901108 32.708354564029...
1236,1658.0,lunch,5,20,48.562212,LINESTRING (-117.1592976256904 32.721978422488...


In [25]:
t4['count'].describe()

count    11269.000000
mean         3.916295
std          4.069166
min          1.000000
25%          1.813853
50%          2.783457
75%          4.454472
max         71.940000
Name: count, dtype: float64

In [26]:
ped[ped.locationuid == '2b8aa3a8'].head()

,timestamp,locationuid,direction,speed,count,tod,dow
6424438,2018-08-29 08:00:00,2b8aa3a8,360,2.641,1,morning,2
6424439,2018-08-29 09:30:00,2b8aa3a8,180,2.641,1,morning,2
6424440,2018-08-29 10:30:00,2b8aa3a8,180,2.641,2,morning,2
6424441,2018-08-29 12:00:00,2b8aa3a8,180,2.641,2,lunch,2
6424442,2018-08-29 12:00:00,2b8aa3a8,360,2.641,1,lunch,2


In [27]:
len(ped)

12747529

In [30]:
t4.groupby('locationuid').sum().sort_values('count', ascending=False)

,direction,speed,count,dow
locationuid,,,,
6b087ad4,6008790,60993.561401,1272288,68905
23c2acf0,6983816,54519.981871,776258,84083
07afd9f2,7198130,62610.996092,695426,82305
08309a7b,2377753,70211.818439,689504,73916
2c8b2e3a,5305304,34546.568933,461365,65533
006da86e,4390269,56106.936872,435034,106043
6bb7900b,4068814,58071.003399,408345,106849
4631f70c,3004915,17793.706020,406421,36893
311ccd0d,7932798,29668.328287,404960,99397
